In [1]:
pip install unidecode

You should consider upgrading via the '/Users/taqtile/Documents/forecast/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [2]:
#data kit
import pandas as pd
import math
from unidecode import unidecode
import numpy as np

#google
from google.cloud import bigquery
from google.oauth2 import service_account


In [3]:
client = bigquery.Client()
query = """

SELECT case when lower(REGEXP_REPLACE(NORMALIZE(brand, NFD), r"\pM", '')) = "yalah falafel" then "yalah" else lower(REGEXP_REPLACE(NORMALIZE(brand, NFD), r"\pM", '')) end as brand FROM EXTERNAL_QUERY("projects/devoro/locations/southamerica-east1/connections/postgres_order", "SELECT name as brand from brand;")


"""
query_brand = client.query(query)

/Users/taqtile/Documents/forecast/venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/taqtile/Documents/forecast/venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For m

In [4]:
df_brand = query_brand.to_dataframe()

In [5]:
braz = pd.Series('braz')
df_brand['brand'] = df_brand['brand'].append(braz, ignore_index=True)

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/3795799057.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_brand['brand'] = df_brand['brand'].append(braz, ignore_index=True)


In [6]:
search = df_brand['brand'].to_list()
search.append('braz')

In [7]:
search

['yalah',
 'devoro',
 'piraja',
 'braz elettrica',
 'braz pizzaria',
 'braz trattoria',
 'ici brasserie',
 'lanchonete da cidade',
 'original',
 'bacio di latte',
 'confeitaria dama',
 'astor',
 'braz']

In [8]:
#Puxando pedidos do store_id 974 (somente Devoro, todos os apps, conforme combinado no dia 2022-03-03)

client = bigquery.Client()
query = """

with products as ( 
select
extract(date from date) as date, origin, e.id, p.sku, lower(REGEXP_REPLACE(NORMALIZE(p.name, NFD), r"\pM", '')) as name
from `devoro.sales.ecommerce_sales` e, unnest(products) p
where store_id = '974' #and origin = 'APP' 
and (client_email !='ClienteGo2GoTeste@gmail.com' or client_email is null) 
group by 1,2,3,4,5
order by 1,2,3,4,5
)

select 
date, origin, id, string_agg(name, '@') produto
from products

group by 1,2,3
order by 1,2,3
"""
query_client = client.query(query)

/Users/taqtile/Documents/forecast/venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/taqtile/Documents/forecast/venv/lib/python3.8/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For m

In [9]:
df_query = query_client.to_dataframe()

In [10]:
df_uber = df_query[df_query['origin'] == 'Uber Eats'].reset_index(drop=True)
df_app = df_query[df_query['origin'] == 'APP'].reset_index(drop=True)
df_rappi = df_query[df_query['origin'] == 'Rappi'].reset_index(drop=True)
df_ifood = df_query[df_query['origin'] == 'iFood'].reset_index(drop=True)

In [11]:
# dict_of_df = {}
# names = []
# for i in df_query['origin'].unique():
#     print(i)
#     dict_of_df["df_{}".format(i).replace(' ', '_')] = df_query[df_query['origin'] == i]
#     names.append("df_{}".format(i).replace(' ', '_'))
    
    

In [12]:
iterator = [df_uber, df_app, df_rappi, df_ifood]

In [13]:
# from mlxtend.frequent_patterns import fpgrowth

# te = TransactionEncoder()
# data_final = pd.DataFrame()


# for j in iterator:
#         origin = j['origin'][0]


#         for i in j['date'].unique():
#             df_iterate = j[j['date'] == i]
#             pedidos = len(df_iterate)

#             #mudando a estrutura dos pedidos para entrar no algoritmo fpgrowth
#             list_of_lists = []

#             for line in df_iterate['produto']:
#                 stripped_line = line.strip('@')
#                 line_list = stripped_line.split('@')
#                 list_of_lists.append(line_list)
#             te_ary = te.fit(list_of_lists).transform(list_of_lists)
#             df = pd.DataFrame(te_ary, columns=te.columns_)

#             #rodando o algoritmo fpgrowth

#             frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)

#             #dividindo o dataframe dos pedidos em 1,2,3,4...n itens
#             frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

#             #transformando a porcentagem de ocorrência em número absoluto de pedidos
#             frequent_itemsets['pedidos'] = frequent_itemsets['support']*pedidos

#             frequent_itemsets['date'] = i 
#             frequent_itemsets['origin'] = origin 
#             data = frequent_itemsets
#             data_final = data_final.append(data,ignore_index=True)


In [14]:
from mlxtend.frequent_patterns import fpgrowth

te = TransactionEncoder()
data_final = pd.DataFrame()


for j in df_query['origin'].unique():
        df_platform = df_query[df_query['origin'] == j].reset_index(drop=True)
        origin = df_platform['origin'][0]


        for i in df_platform['date'].unique():
            df_iterate = df_platform[df_platform['date'] == i]
            pedidos = len(df_iterate)

            #mudando a estrutura dos pedidos para entrar no algoritmo fpgrowth
            list_of_lists = []

            for line in df_iterate['produto']:
                stripped_line = line.strip('@')
                line_list = stripped_line.split('@')
                list_of_lists.append(line_list)
            te_ary = te.fit(list_of_lists).transform(list_of_lists)
            df = pd.DataFrame(te_ary, columns=te.columns_)

            #rodando o algoritmo fpgrowth

            frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)

            #dividindo o dataframe dos pedidos em 1,2,3,4...n itens
            frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

            #transformando a porcentagem de ocorrência em número absoluto de pedidos
            frequent_itemsets['pedidos'] = frequent_itemsets['support']*pedidos

            frequent_itemsets['date'] = i 
            frequent_itemsets['origin'] = origin 
            data = frequent_itemsets
            data_final = data_final.append(data,ignore_index=True)


/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

...

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)
/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2962107175.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data_final.append(data,ignore_index=True)


In [15]:
data_final

,support,itemsets,length,pedidos,date,origin
0,0.200000,(prato do dia - 30% off - picadinho - bar astor),1,1.0,2021-07-26,Uber Eats
1,0.200000,(saladinha executiva - piraja),1,1.0,2021-07-26,Uber Eats
2,0.200000,(pao braz trattoria - inteiro),1,1.0,2021-07-26,Uber Eats
3,0.200000,(caldinho de feijao - piraja),1,1.0,2021-07-26,Uber Eats
4,0.200000,(file a parmegiana - piraja),1,1.0,2021-07-26,Uber Eats
...,...,...,...,...,...,...
119157,0.038462,"(calabrese piccante - braz elettrica, smoked -...",2,1.0,2022-03-27,Rappi
119158,0.038462,"(mousse chocolate ici pequeno, smoked - lancho...",2,1.0,2022-03-27,Rappi
119159,0.038462,"(mousse chocolate ici pequeno, calabrese picca...",3,1.0,2022-03-27,Rappi
119160,0.038462,"(executivo kafta - yalah, mix petiscos - astor...",2,1.0,2022-03-27,Rappi


In [16]:
data_final["itemsets"] = data_final["itemsets"].apply(lambda x: list(x)[0:15]).astype("unicode")



In [17]:
data_final['itemsets'] = data_final['itemsets'].str.replace("[", "").str.replace("]", "").str.replace("'", "")

data_final["marcas"] = data_final['itemsets'].str.findall('(' + '|'.join(search) + ')')



/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/3868272196.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_final['itemsets'] = data_final['itemsets'].str.replace("[", "").str.replace("]", "").str.replace("'", "")


In [18]:
data_final["marcas"] = data_final['marcas'].astype(str).str.replace("[", "").str.replace("]", "").str.replace("'", "")

/var/folders/4n/5zbxw_xn33bdy9f3rndb3h1c0000gp/T/ipykernel_26782/2247901126.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_final["marcas"] = data_final['marcas'].astype(str).str.replace("[", "").str.replace("]", "").str.replace("'", "")


In [19]:
data_final

,support,itemsets,length,pedidos,date,origin,marcas
0,0.200000,prato do dia - 30% off - picadinho - bar astor,1,1.0,2021-07-26,Uber Eats,astor
1,0.200000,saladinha executiva - piraja,1,1.0,2021-07-26,Uber Eats,piraja
2,0.200000,pao braz trattoria - inteiro,1,1.0,2021-07-26,Uber Eats,braz trattoria
3,0.200000,caldinho de feijao - piraja,1,1.0,2021-07-26,Uber Eats,piraja
4,0.200000,file a parmegiana - piraja,1,1.0,2021-07-26,Uber Eats,piraja
...,...,...,...,...,...,...,...
119157,0.038462,"calabrese piccante - braz elettrica, smoked - ...",2,1.0,2022-03-27,Rappi,"braz elettrica, lanchonete da cidade"
119158,0.038462,"mousse chocolate ici pequeno, smoked - lanchon...",2,1.0,2022-03-27,Rappi,lanchonete da cidade
119159,0.038462,"mousse chocolate ici pequeno, calabrese piccan...",3,1.0,2022-03-27,Rappi,"braz elettrica, lanchonete da cidade"
119160,0.038462,"executivo kafta - yalah, mix petiscos - astor ...",2,1.0,2022-03-27,Rappi,"yalah, astor, piraja, original"


In [20]:
data_final.to_csv("mix_2022-03-28.csv")

array(['astor', 'piraja', 'braz trattoria', ...,
       'ici brasserie, piraja, braz trattoria, braz trattoria, braz elettrica',
       'braz elettrica, ici brasserie, braz trattoria, braz trattoria, braz elettrica',
       'braz elettrica, ici brasserie, piraja, braz trattoria, braz elettrica'],
      dtype=object)

In [232]:
for i in df_query['origin'].unique():
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [239]:
pip install dtale

     |████████████████████████████████| 12.7 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 642 kB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 9.6 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 23.7 MB/s eta 0:00:01
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
     |████████████████████████████████| 62 kB 2.7 MB/s  eta 0:00:01
     |████████████████████████████████| 870 kB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 85.2 MB 6.7 MB/s eta 0:00:011
     |████████████████████████████████| 421 kB 26.1 MB/s eta 0:00:01
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-py3-none-any.whl size=62590 sha256=7f64425124c3bdcb5

  Attempting uninstall: missingno
    Found existing installation: missingno 0.5.0
    Uninstalling missingno-0.5.0:
      Successfully uninstalled missingno-0.5.0
You should consider upgrading via the '/Users/taqtile/Documents/forecast/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [240]:
import dtale

In [244]:
data_final

,support,itemsets,length,pedidos,date,origin,marcas
0,0.200000,prato do dia - 30% off - picadinho - bar astor,1,1.0,2021-07-26,Uber Eats,astor
1,0.200000,saladinha executiva - piraja,1,1.0,2021-07-26,Uber Eats,piraja
2,0.200000,pao braz trattoria - inteiro,1,1.0,2021-07-26,Uber Eats,braz trattoria
3,0.200000,caldinho de feijao - piraja,1,1.0,2021-07-26,Uber Eats,piraja
4,0.200000,file a parmegiana - piraja,1,1.0,2021-07-26,Uber Eats,piraja
...,...,...,...,...,...,...,...
110457,0.076923,"x-salada - lanchonete da cidade, mix petiscos ...",4,1.0,2022-03-10,Rappi,"lanchonete da cidade, astor, piraja, original,..."
110458,0.076923,"pastel de queijo do astor, mix petiscos - asto...",4,1.0,2022-03-10,Rappi,"astor, astor, piraja, original, lanchonete da ..."
110459,0.076923,"pastel de queijo do astor, mix petiscos - asto...",4,1.0,2022-03-10,Rappi,"astor, astor, piraja, original, ici brasserie,..."
110460,0.076923,"pastel de queijo do astor, x-salada - lanchone...",4,1.0,2022-03-10,Rappi,"astor, lanchonete da cidade, ici brasserie, yalah"


In [245]:
dtale.show(data_final)